In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
import seaborn as sns
from sklearn.metrics import mean_squared_error

resultado_anterior = 0
maximo = 0

# Función para obtener los labels

In [10]:
def calculate_label(reference_df, df_labels, id_feature_name, reference_date, label_name, date_feature_name):
    max_time_window = 259200 #es el tiempo maximo que se da en 3 dias
    date = pd.to_datetime(reference_date)
    df_result = df_labels.groupby(id_feature_name).agg({date_feature_name : 'min'}).reset_index()
    df_result[label_name] = (df_result[date_feature_name] - date).dt.total_seconds()
    del df_result[date_feature_name]
    merge_df = reference_df[[id_feature_name]].merge(df_result, how = 'left', on = id_feature_name).fillna(max_time_window)
    return merge_df

Labels de subastas. Segundos hasta que el usuario vuelva a aparecer en una subasta

In [ ]:
labels_auc_18_20 = calculate_label(auctions_18_20, auctions_21_23, "device_id", "2019-04-18", "secs_to_appear", "date")
labels_auc_19_21 = calculate_label(auctions_19_21, auctions_22_24, "device_id", "2019-04-19", "secs_to_appear", "date")
labels_auc_20_22 = calculate_label(auctions_20_22, auctions_23_25, "device_id", "2019-04-20", "secs_to_appear", "date")
labels_auc_21_23 = calculate_label(auctions_21_23, auctions_24_26,"device_id", "2019-04-21", "secs_to_appear", "date")
#labels_auc_22_24 = calculate_label(auctions_22_24, auctions_25_27,"device_id", "2019-04-22", "secs_to_appear", "date")
#como hacemos en la ventana del 22-24? no tenemos los datos del 27

Labels de installs. Segundos hasta que el usuario vuelva a convertir

In [11]:
labels_inst_18_20 = calculate_label(installs_18_20, installs_21_23, "ref_hash",  "2019-04-18", "seconds_to_install", "created")
labels_inst_19_21 = calculate_label(installs_19_21, installs_22_24, "ref_hash",  "2019-04-19", "seconds_to_install", "created")
labels_inst_20_22 = calculate_label(installs_20_22, installs_23_25, "ref_hash",  "2019-04-20", "seconds_to_install", "created")
labels_inst_21_23 = calculate_label(installs_21_23, installs_24_26, "ref_hash",  "2019-04-21", "seconds_to_install", "created")
#labels_inst_22_24 = calculate_label(installs_22_24, "ref_hash",  "2019-04-22", "seconds_to_install", "created")

In [30]:
labels_inst_18_20.head(3)

,ref_hash,seconds_to_install
0,5230323462636548010,259200.0
1,5097163995161606833,259200.0
2,6328027616411983332,259200.0


# Carga de los df de features para entrenar

In [45]:
#auc_18_20_features = pd.read_csv("") ...

# Funciones de entrenamiento

In [ ]:
def guardar_submit(params, result):
    tiempo = "time"
    with open("historial_submits.txt","a+") as f:
        f.write("\n"+tiempo+"|"+params+"|")

In [ ]:
def entrenar_modelo(params):
    result = None#entrenar(params)
    guardar_submit(params, result)

In [ ]:
def modelfit(model, features, labels, cv_folds=5, early_stopping_rounds=50):
    parametros = model.get_xgb_params()
    dmatrix = xgb.DMatrix(features.values, label=labels.values)
    cvresult = xgb.cv(parametros, dmatrix, num_boost_round=model.get_params()['n_estimators'],
                      nfold=cv_folds,metrics='auc', #verbose_eval=True,
                      early_stopping_rounds=early_stopping_rounds)
    model.set_params(n_estimators=cvresult.shape[0])
    model.fit(features, labels,eval_metric='auc')

In [ ]:
def entrenar(df_features, labels):
    modelo = xgb.XGBRegressor(
        #booster='dart',
        learning_rate =0.01,
        n_estimators=1000,
        max_depth=4,
        min_child_weight=8,
        gamma=0.22,
        subsample=1,
        colsample_bytree=1,
        objective= 'binary:logistic',
        nthread=-1,
        scale_pos_weight=18.8,
        #sample_type='weighted',
        #rate_drop=0.1,
        #skip_dropout=0.5,
        random_state=272
    )
    
    df_features = df_features.join(labels, how = 'inner')
    X, y = df_features.iloc[:,:-1], df_features.iloc[:,-1:]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
    
    modelfit(modelo, X_train, y_train, early_stopping_rounds=30)

    predsa = modelo.predict(X_test)
    resultado = roc_auc_score(y_test, predsa)

# Entrenamiento

# Submit to Kaggle

Las predicciones tendrán seteadas como índice los ref_hash para no perder la referencia
No es necesario filtrar los ref_hash y quedarnos solo con los target en las predicciones que obtenemos ya que de eso
se encarga la función 

In [112]:
example_submit = labels_inst_18_20.set_index('ref_hash')
example_submit.head(2)

,obj
ref_hash,
5230323462636548010,259200.0
5097163995161606833,259200.0


In [111]:
targets = pd.read_csv('target_final_competencia_revamped.csv')
targets.head(1)

,ref_hash,obj
0,1000169251625791232_sc,0


In [135]:
def create_submit_df(auctions_predictions, installs_predictions, target):
    
    target = target.set_index('ref_hash')
    
    auc = auctions_predictions.reset_index()
    auc.columns = ['ref_hash','obj']
    auc['ref_hash'] = auc['ref_hash'].astype(str) + "_sc"
    auc = auc.set_index('ref_hash')
    
    ins = installs_predictions.reset_index()
    ins.columns = ['ref_hash','obj']
    ins['ref_hash'] = ins['ref_hash'].astype(str) + "_st"
    ins = ins.set_index('ref_hash')
    
    frames = [ins,auc]
    submit_result = pd.concat(frames).reset_index()
    target_list = target.reset_index('ref_hash')['ref_hash'].tolist()
    return submit_result.loc[submit_result['ref_hash'].isin(target_list)].sort_values(by = 'ref_hash')

In [136]:
kaggle_sub = create_submit_df(labels_inst_18_20.set_index('ref_hash'), labels_inst_18_20.set_index('ref_hash'), targets)

In [137]:
kaggle_sub.head(2)

,ref_hash,obj
204098,12977391562678610_sc,259200.0
43237,12977391562678610_st,259200.0
